In [2]:
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Lasso, HuberRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, PowerTransformer, FunctionTransformer, StandardScaler
from sklearn.svm import LinearSVR, SVR
from xgboost import XGBRegressor

from utils.ml_data_provider import SectoralDataProvider

In [3]:
data_provider = SectoralDataProvider(cache_path='/Users/j4yzer/PycharmProjects/VKR/data/sectoral_ml')
data : pd.DataFrame = data_provider.load_data()
data['nextRelativeToSPReturn'] = data['relativeToSPReturn'].shift(-1)
# data = data.set_index('date')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20675 entries, 0 to 20674
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ticker                                   20675 non-null  object 
 1   date                                     20675 non-null  object 
 2   sector                                   20675 non-null  object 
 3   sectoralIndex                            20675 non-null  object 
 4   absoluteReturn                           19209 non-null  float64
 5   relativeToSPReturn                       19209 non-null  float64
 6   relativeToSectoralIndexReturn            19209 non-null  float64
 7   nextPeriodRelativeToSectoralIndexReturn  19209 non-null  float64
 8   closePrice                               20675 non-null  float64
 9   dilutedEarningsPerShare                  20675 non-null  float64
 10  freeCashFlowPerShare                     20675

,absoluteReturn,relativeToSPReturn,relativeToSectoralIndexReturn,nextPeriodRelativeToSectoralIndexReturn,closePrice,dilutedEarningsPerShare,freeCashFlowPerShare,bookValuePerShare,equityRatio,marketCap,...,priceToBookRatio,enterpriseValueToSalesRatio,currentRatio,roic,netCurrentAssetValuePerShare,ebitdaMargin,netIncomeMargin,returnOnAssets,returnOnEquity,nextRelativeToSPReturn
count,1.920900e+04,1.920900e+04,1.920900e+04,1.920900e+04,20675.000000,2.067500e+04,20675.000000,20675.000000,20675.000000,2.067500e+04,...,20675.000000,20675.000000,20675.000000,20675.000000,2.067500e+04,20675.000000,20675.000000,20675.000000,20675.000000,1.920900e+04
mean,inf,inf,inf,inf,48.993744,NaN,0.547413,28.182981,0.432702,1.922698e+10,...,2.615234,7.888780,2.452396,0.084762,-inf,0.064115,-0.186277,0.051213,0.117414,inf
std,NaN,NaN,NaN,NaN,63.675534,NaN,42.691063,178.631231,0.217576,3.917058e+10,...,64.835246,379.846695,10.321225,0.772190,NaN,12.290531,17.683512,0.205015,2.513563,NaN
min,-1.000000e+00,-9.505327e-01,-9.726245e-01,-9.726245e-01,0.000000,-inf,-3049.391300,-1021.364200,-2.040700,0.000000e+00,...,-8717.450000,-338.000000,0.000000,-60.000000,-inf,-1395.000000,-2014.000000,-3.254600,-257.973600,-9.505327e-01
25%,-6.722966e-02,-8.005409e-02,-7.233059e-02,-7.233059e-02,15.295000,8.712618e-02,-0.035600,7.106450,0.325500,1.906255e+09,...,1.186900,0.879300,1.104600,0.021800,-2.117023e+01,0.115000,0.022100,0.016450,0.037850,-8.005409e-02
50%,3.159665e-02,5.274651e-03,5.118437e-03,5.118437e-03,32.280000,4.273292e-01,0.344200,14.594900,0.434200,6.308645e+09,...,2.084600,1.753300,1.592900,0.074300,-8.463073e+00,0.194000,0.076200,0.052500,0.125800,5.274651e-03
75%,1.392366e-01,9.811018e-02,9.195576e-02,9.195576e-02,60.050000,1.000837e+00,1.031600,25.952450,0.552150,1.976868e+10,...,3.552700,3.177400,2.371900,0.133200,-1.483547e+00,0.345200,0.143400,0.090700,0.217700,9.811018e-02
max,inf,inf,inf,inf,1777.230000,inf,2217.021200,5743.304300,1.000000,1.041448e+12,...,926.276800,42707.400000,945.062000,45.846100,9.749637e+01,219.000000,136.666600,17.854400,97.272700,inf


In [4]:
data = data.replace([-np.Inf, np.Inf], np.nan)
data = data.dropna()
# data.info()
# data.describe()

In [16]:
def drop_outliers_iqr(df, iqr_bound=(0.25, 0.75), IQR_k=7):
    Q1 = df.quantile(iqr_bound[0])
    Q3 = df.quantile(iqr_bound[1])
    IQR = Q3 - Q1
    return df[~((df < (Q1 - IQR_k * IQR)) | (df > (Q3 + IQR_k * IQR))).any(axis=1)]
def drop_outliers_quantile(df, keep_inside_quantiles_bounds = (0.01, 0.99)):
    q_l = df.quantile(keep_inside_quantiles_bounds[0])
    q_u = df.quantile(keep_inside_quantiles_bounds[1])
    print(keep_inside_quantiles_bounds)
    return df[~((df < q_l) | (df > q_u)).any(axis=1)]

def visualise_data(df: pd.DataFrame, remove_outliers=True, remove_outliers_function=drop_outliers_quantile, remove_outliers_kwargs = None):
    df_to_visualize = df.copy()
    if remove_outliers:
        df_to_visualize = remove_outliers_function(df_to_visualize, **remove_outliers_kwargs)
        print(df.index.size - df_to_visualize.index.size)
    df_to_visualize.hist(bins=50, figsize=(20, 15))
    fig, axes = plt.subplots(5, int(np.ceil(len(df_to_visualize.columns) / 5)), figsize=(20, 15))
    for i,el in enumerate(list(df_to_visualize.columns.values)):
        df_to_visualize.boxplot([el], ax=axes.flatten()[i])
    plt.tight_layout()
    plt.show()
def eval(model, features, labels):
    predictions = model.predict(features)
    errors = abs(predictions - labels)
    mape = 100 * np.mean(errors / labels)
    accuracy = 100 - mape
    print('Model Performance')
    stats = pd.DataFrame([('RMSE', mean_squared_error(labels, predictions, squared=False)),(
                  'MAE', mean_absolute_error(labels, predictions)),(
                  'R2', r2_score(labels, predictions))], columns=['Статистика', 'Значение'])
    print(stats)
    return stats

In [6]:
train_data, test_data = train_test_split(data.drop(['ticker', 'sector', 'sectoralIndex', 'date', 'nextPeriodRelativeToSectoralIndexReturn'], axis=1), test_size=0.2, shuffle=False)

# visualise_data(train_data, remove_outliers=False)
# visualise_data(train_data, remove_outliers=True, remove_outliers_function=drop_outliers_iqr, remove_outliers_kwargs={'IQR_k': 30})

In [7]:
train_data = drop_outliers_iqr(train_data, IQR_k=30)
test_data = drop_outliers_iqr(test_data, IQR_k=30)

r_scaler = RobustScaler()
p_scaler = PowerTransformer()

long_tail_scaler = make_pipeline(p_scaler)
normal_scaler = make_pipeline(r_scaler)


y_train_data = train_data['nextRelativeToSPReturn']
X_train_data = train_data.drop(['nextRelativeToSPReturn'], axis=1)

long_tail_columns = ['closePrice', 'bookValuePerShare', 'marketCap', 'priceToSalesRatio', 'enterpriseValueToSalesRatio', 'currentRatio', 'netCurrentAssetValuePerShare']
normal_columns = [k for k in X_train_data.columns if k not in long_tail_columns]
prep_pipeline = ColumnTransformer([('lt', long_tail_scaler, long_tail_columns), ('normal', normal_scaler, normal_columns)])

X_train_scaled = pd.DataFrame(prep_pipeline.fit_transform(X=X_train_data), columns=X_train_data.columns, index=X_train_data.index)

# visualise_data(X_train_scaled, remove_outliers=False)

In [9]:
def get_cross_val_row(cross_val_res, stat, algo, stat_metrics=['mean', 'std']):
    stat_map = {'r2': 'R2', 'neg_mean_absolute_error': 'MAE', 'neg_root_mean_squared_error': 'RMSE'}
    stat = stat_map[stat]
    stats = pd.Series(cross_val_res).describe().to_frame('Значение').reset_index().rename({'index': 'Показатель статистики'}, axis=1)
    stats.insert(0, 'Статистика', value=stat)
    stats.insert(0, 'Алгоритм', value=algo)
    return stats[stats['Показатель статистики'].isin(stat_metrics)]
scorings = ['r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error']
y_test_data = test_data['nextRelativeToSPReturn']
X_test_data = test_data.drop(['nextRelativeToSPReturn'], axis=1)

linear_regr_pipeline = make_pipeline(prep_pipeline, LinearRegression())
svr_pipeline = make_pipeline(prep_pipeline, SVR(C=1.2, gamma=1e-5))

random_forest_pipeline = make_pipeline(prep_pipeline, RandomForestRegressor(max_features=4, n_estimators=300, bootstrap=True,random_state=42))
huber_pipeline = make_pipeline(prep_pipeline, HuberRegressor())

pipelines = [('Linear regr', linear_regr_pipeline), ('SVR', svr_pipeline)
    , ('Random forest', random_forest_pipeline)]
methods_table = pd.DataFrame(columns=['Алгоритм', 'Статистика','Показатель статистики', 'Значение'])
for name, pipeline in pipelines:
        print(name)
        for scoring in scorings:
            scrs = -cross_val_score(pipeline, X_train_data, y_train_data,
                                      scoring=scoring, cv=5)
            print(pd.Series(scrs).describe())
            methods_table = pd.concat([methods_table, get_cross_val_row(scrs, scoring, name)])

        est = pipeline.fit(X_train_data, y_train_data)

        print('train: ')
        eval(est, X_train_data, y_train_data)
        print('test: ')
        eval(est, X_test_data, y_test_data)
out = pd.Series(methods_table['Значение'].array, index=pd.MultiIndex.from_frame(methods_table.drop('Значение', axis=1))).to_frame('Значение')
print(out)
# out.to_excel()

In [16]:
lasso_feat_selection = SelectFromModel(estimator=Lasso(alpha=1.13))
rf_feat_selection = SelectFromModel(estimator=RandomForestRegressor(max_features=4, n_estimators=800, bootstrap=True))

print(lasso_feat_selection.fit(X=X_train_data, y=y_train_data).get_feature_names_out())
print(rf_feat_selection.fit(X=X_train_data, y=y_train_data).get_feature_names_out())

[]
['absoluteReturn' 'relativeToSPReturn' 'relativeToSectoralIndexReturn'
 'freeCashFlowPerShare' 'marketCap' 'currentRatio'
 'netCurrentAssetValuePerShare' 'ebitdaMargin']


In [11]:
def get_cross_val_row(cross_val_res, stat, algo, stat_metrics=['mean', 'std']):
    stat_map = {'r2': 'R2', 'neg_mean_absolute_error': 'MAE', 'neg_root_mean_squared_error': 'RMSE'}
    stat = stat_map[stat]
    stats = pd.Series(cross_val_res).describe().to_frame('Значение').reset_index().rename({'index': 'Показатель статистики'}, axis=1)
    stats.insert(0, 'Статистика', value=stat)
    stats.insert(0, 'Алгоритм', value=algo)
    return stats[stats['Показатель статистики'].isin(stat_metrics)]
scorings = ['r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error']
selected_features = ['absoluteReturn', 'relativeToSPReturn', 'relativeToSectoralIndexReturn',
 'freeCashFlowPerShare', 'marketCap', 'currentRatio', 'netCurrentAssetValuePerShare', 'ebitdaMargin', 'closePrice']
X_train_selected = X_train_data[selected_features]
X_test_selected = X_test_data[selected_features]

long_tail_columns = ['closePrice', 'marketCap', 'currentRatio', 'netCurrentAssetValuePerShare']
normal_columns = [k for k in X_train_selected.columns if k not in long_tail_columns]



prep_pipeline = ColumnTransformer([('lt', long_tail_scaler, long_tail_columns), ('normal', normal_scaler, normal_columns)])

linear_regr_pipeline = make_pipeline(prep_pipeline, LinearRegression())
svr_pipeline = make_pipeline(prep_pipeline, SVR(C=1.2, gamma=1e-5))
xgb_pipeline = make_pipeline(prep_pipeline, XGBRegressor(objective='reg:squarederror'))

random_forest_pipeline = make_pipeline(prep_pipeline, RandomForestRegressor(max_features=4, n_estimators=300, bootstrap=True,random_state=42))

pipelines = [('Linear regr', linear_regr_pipeline), ('SVR', svr_pipeline)
    , ('Random forest', random_forest_pipeline)]
methods_table = pd.DataFrame(columns=['Алгоритм', 'Статистика','Показатель статистики', 'Значение'])
for name, pipeline in pipelines:
        print(name)
        for scoring in scorings:
            scrs = -cross_val_score(pipeline, X_train_selected, y_train_data,
                                      scoring=scoring, cv=5)
            print(pd.Series(scrs).describe())
            methods_table = pd.concat([methods_table, get_cross_val_row(scrs, scoring, name)])

        est = pipeline.fit(X_train_selected, y_train_data)

        print('train: ')
        eval(est, X_train_selected, y_train_data)
        print('test: ')
        eval(est, X_test_selected, y_test_data)
out = pd.Series(methods_table['Значение'].array, index=pd.MultiIndex.from_frame(methods_table.drop('Значение', axis=1))).to_frame('Значение')
print(out)
out.to_excel('out_table_global_tuned_cv.xlsx')

Linear regr
count    5.000000
mean    -0.008539
std      0.021271
min     -0.038894
25%     -0.019649
50%     -0.003536
75%      0.003565
max      0.015820
dtype: float64
count    5.000000
mean     0.118830
std      0.014943
min      0.103863
25%      0.109854
50%      0.115359
75%      0.122518
max      0.142555
dtype: float64
count    5.000000
mean     0.173034
std      0.028145
min      0.141580
25%      0.149430
50%      0.177095
75%      0.186181
max      0.210886
dtype: float64
train: 
Model Performance
  Статистика             Значение
0        MSE  0.17394711628821463
1        MAE   0.1182336769403626
2         R2    0.040005274807554
test: 
Model Performance
  Статистика             Значение
0        MSE   0.2548899671442626
1        MAE  0.16311625286851486
2         R2  0.07217930084341573
SVR
count    5.000000
mean     0.018215
std      0.030748
min     -0.007748
25%     -0.005537
50%      0.010519
75%      0.026473
max      0.067369
dtype: float64
count    5.000000
mean   

In [17]:
methods_table_for_sector = pd.DataFrame(columns=['Алгоритм', 'Сет', 'Статистика', 'Значение'])
for name, pipeline in pipelines:
    print(name)

    est = pipeline.fit(X_train_data, y_train_data)

    print('train: ')
    stats_train = eval(est, X_train_data, y_train_data)
    stats_train.insert(0, 'Сет', 'train')
    print('test: ')
    stats_test = eval(est, X_test_data, y_test_data)
    stats_test.insert(0, 'Сет', 'test')

    stats_for_algo = pd.concat([stats_train, stats_test])
    stats_for_algo.insert(0, 'Алгоритм', name)
    methods_table_for_sector = pd.concat([methods_table_for_sector, stats_for_algo])


out = pd.Series(methods_table_for_sector['Значение'].array,
                index=pd.MultiIndex.from_frame(methods_table_for_sector.drop('Значение', axis=1))).to_frame(
    'Значение')
out.to_excel('out_table_global_tuned_train_test.xlsx')

Linear regr
train: 
Model Performance
  Статистика  Значение
0       RMSE  0.173947
1        MAE  0.118234
2         R2  0.040005
test: 
Model Performance
  Статистика  Значение
0       RMSE  0.254890
1        MAE  0.163116
2         R2  0.072179
SVR
train: 
Model Performance
  Статистика  Значение
0       RMSE  0.176441
1        MAE  0.119000
2         R2  0.012281
test: 
Model Performance
  Статистика  Значение
0       RMSE  0.262187
1        MAE  0.166123
2         R2  0.018299
Random forest
train: 
Model Performance
  Статистика  Значение
0       RMSE  0.065545
1        MAE  0.044912
2         R2  0.863693
test: 
Model Performance
  Статистика  Значение
0       RMSE  0.255741
1        MAE  0.165713
2         R2  0.065972
